Python simulation of paper: A Phase-Type Based Markov Chain Model for IEEE 802.16e Sleep Mode and Its Performance Analysis

In [4]:
from scipy import integrate
import math
from collections import deque
import random

In [2]:
def f(t):
    return 0.25 * math.exp(-0.25*t)
v, err = integrate.quad(f, 0, float('inf'))
print(v, err)

def f1(t):
    return t*0.25 * math.exp(-0.25*t)
v, err = integrate.quad(f1, 0, float('inf'))
print(v)

1.0000000000000002 2.8388779836045405e-11
3.9999999999999996


In [19]:
# listening interval (ms)
tl = 5

# initial sleep window (ms)
tb = 40

#final sleep window exponent
M = 5

# service rate (packets/second) 
mu = 25

# buffer size (packets)
N = 5

# traffic load = lambda / mu
trafficLoad = 0.1

# arrival rate (packets/second) avoide using lambda because it is a reserved keyword of python
lamda = trafficLoad * mu

# total time (hour)
hour = 50

In [19]:
sm = 0
cnt = 90000
for i in range(cnt):
    r = random.expovariate(0.25)
#     print(r)
    sm += r
print(sm / cnt)

3.9860327525953347


In [21]:
packetArrival = deque()
cur = 0
while cur < 1000 * 60 * 60 * hour * 1.1:
    nxt = random.expovariate(1/(1/lamda*1000))
    cur += nxt
    packetArrival.append(cur)
print(len(packetArrival))

494705


In [4]:
from scipy import integrate
import math
from collections import deque
import random

# listening interval (ms)
tl = 5
# initial sleep window (ms)
tb = 40
#final sleep window exponent
M = 5
# service rate (packets/second) 
mu = 25
# buffer size (packets)
N = 10
# traffic load = lambda / mu
trafficLoad = 0.2
# arrival rate (packets/second) avoide using lambda because it is a reserved keyword of python
lamda = trafficLoad * mu
# total time (hour)
hour = 50

packetArrival = deque()
cur = 0
while cur < 1000 * 60 * 60 * hour * 1.1:
    nxt = random.expovariate(1/(1/lamda*1000))
    cur += nxt
    packetArrival.append(cur)
print(len(packetArrival))

buffer = deque()
cnt = 0
totalWait = 0
sleepState = 0
cur = 0
while cur < 60 * 60 * 1000 * hour:
    cur += tb * (2 ** sleepState) + tl
    while packetArrival[0] <= cur:
        buffer.append(packetArrival.popleft())
#         if len(buffer) > N:
#             buffer.pop()
    if len(buffer) == 0:
        sleepState = min(sleepState+1, M)
    else:
        while len(buffer) != 0:
            cnt += 1
            cur += 1000 / mu
            totalWait += cur - buffer.popleft()
            while packetArrival[0] <= cur:
                buffer.append(packetArrival.popleft())
#                 if len(buffer) > N:
#                     buffer.pop()
        sleepState = 0
print(totalWait / cnt)

988355
156.25777992849726


In [5]:
def simulate(trafficLoad):
    # listening interval (ms)
    tl = 5
    # initial sleep window (ms)
    tb = 40
    #final sleep window exponent
    M = 5
    # service rate (packets/second) 
    mu = 25
    # buffer size (packets)
    N = 10
    # arrival rate (packets/second) avoide using lambda because it is a reserved keyword of python
    lamda = trafficLoad * mu
    # total time (hour)
    hour = 50

    packetArrival = deque()
    cur = 0
    while cur < 1000 * 60 * 60 * hour * 1.1:
        nxt = random.expovariate(1/(1/lamda*1000))
        cur += nxt
        packetArrival.append(cur)
#     print(len(packetArrival))

    buffer = deque()
    cnt = 0
    totalWait = 0
    sleepState = 0
    cur = 0
    while cur < 60 * 60 * 1000 * hour:
        cur += tb * (2 ** sleepState) + tl
        while packetArrival[0] <= cur:
            buffer.append(packetArrival.popleft())
    #         if len(buffer) > N:
    #             buffer.pop()
        if len(buffer) == 0:
            sleepState = min(sleepState+1, M)
        else:
            while len(buffer) != 0:
                cnt += 1
                cur += 1000 / mu
                totalWait += cur - buffer.popleft()
                while packetArrival[0] <= cur:
                    buffer.append(packetArrival.popleft())
    #                 if len(buffer) > N:
    #                     buffer.pop()
            sleepState = 0
    return totalWait / cnt

In [ ]:
import math
import numpy as np
def theoretical_calculate(trafficLoad):
    mu = 0.025
    Ev = 1/mu
    trafficload = 0.2
    lamda = mu * trafficload
    M = 5
    N = 10
    Tb = 40
    Tl = 5